## Imports

In [1]:
import os
from typing import List
import json
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil
import sys
import logging 

logging.basicConfig(
     level=logging.INFO, 
     format= '[%(asctime)s|%(levelname)s|%(module)s.py:%(lineno)s] %(message)s',
     datefmt='%H:%M:%S'
 )
import tqdm.notebook as tq
from tqdm import tqdm
# Create new `pandas` methods which use `tqdm` progress
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()

from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, 
    TrainingArguments, Trainer, EarlyStoppingCallback, IntervalStrategy
)

from defi_textmine_2025.data import TARGET_COL, INTERIM_DIR, MODELS_DIR, submission_path

/home/tagny/.pyenv/versions/3.11.9/envs/kaggle/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[15:55:09|INFO|config.py:58] PyTorch version 2.3.1 available.
[15:55:09|INFO|config.py:105] TensorFlow version 2.16.2 available.
2024-07-14 15:55:11.564598: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-14 15:55:11.587989: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-14 15:55:11.588035: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugi

In [2]:
RANDOM_SEED = 123  # random reproducibility
np.random.seed(RANDOM_SEED)
# BASE_CHECKPOINT = "bert-base-uncased"
BASE_CHECKPOINT = "bert-base-multilingual-cased"
# BASE_CHECKPOINT = "camembert/camembert-base"
TASK_NAME = "hasrelation_btc"
TASK_TARGET_COL = "label" # hasrelation?

entity_classes = {'TERRORIST_OR_CRIMINAL', 'LASTNAME', 'LENGTH', 'NATURAL_CAUSES_DEATH', 'COLOR', 'STRIKE', 'DRUG_OPERATION', 'HEIGHT', 'INTERGOVERNMENTAL_ORGANISATION', 'TRAFFICKING', 'NON_MILITARY_GOVERNMENT_ORGANISATION', 'TIME_MIN', 'DEMONSTRATION', 'TIME_EXACT', 'FIRE', 'QUANTITY_MIN', 'MATERIEL', 'GATHERING', 'PLACE', 'CRIMINAL_ARREST', 'CBRN_EVENT', 'ECONOMICAL_CRISIS', 'ACCIDENT', 'LONGITUDE', 'BOMBING', 'MATERIAL_REFERENCE', 'WIDTH', 'FIRSTNAME', 'MILITARY_ORGANISATION', 'CIVILIAN', 'QUANTITY_MAX', 'CATEGORY', 'POLITICAL_VIOLENCE', 'EPIDEMIC', 'TIME_MAX', 'TIME_FUZZY', 'NATURAL_EVENT', 'SUICIDE', 'CIVIL_WAR_OUTBREAK', 'POLLUTION', 'ILLEGAL_CIVIL_DEMONSTRATION', 'NATIONALITY', 'GROUP_OF_INDIVIDUALS', 'QUANTITY_FUZZY', 'RIOT', 'WEIGHT', 'THEFT', 'MILITARY', 'NON_GOVERNMENTAL_ORGANISATION', 'LATITUDE', 'COUP_D_ETAT', 'ELECTION', 'HOOLIGANISM_TROUBLEMAKING', 'QUANTITY_EXACT', 'AGITATING_TROUBLE_MAKING'}

generated_data_dir_path = os.path.join(INTERIM_DIR, "multilabel_tagged_text_dataset")
assert os.path.exists(generated_data_dir_path)
train_dir = os.path.join(generated_data_dir_path, "train")
test_dir = os.path.join(generated_data_dir_path, "test")

preprocessed_data_dir = os.path.join(INTERIM_DIR, "one_hot_multilabel_tagged_text_dataset")
labeled_preprocessed_data_dir_path = os.path.join(preprocessed_data_dir,"train")
! mkdir -p {labeled_preprocessed_data_dir_path}

model_dir_path = os.path.join(MODELS_DIR, f"finetuned-{BASE_CHECKPOINT}")
! mkdir -p {model_dir_path}
model_dict_state_path = os.path.join(model_dir_path,"MLTC_model_state.bin")


In [3]:
def get_cat_var_distribution(cat_var: pd.Series) -> pd.DataFrame:
    return pd.concat(
        [cat_var.value_counts(), cat_var.value_counts(normalize=True)], axis=1
    )

## Prepare the datasets for the binary text classification

### Load and process the target

In [4]:
def load_csv(dir_or_file_path: str, index_col=None, sep=',') -> pd.DataFrame:
    if os.path.isdir(dir_or_file_path):
        all_files = glob.glob(os.path.join(dir_or_file_path , "*.csv"))  
    else:
        assert dir_or_file_path.endswith(".csv")
        all_files = [dir_or_file_path]
    assert len(all_files) > 0
    return pd.concat([pd.read_csv(filename, index_col=index_col, header=0, sep=sep) for filename in all_files], axis=0, ignore_index=True)

train_df = load_csv(train_dir).assign(**{TASK_TARGET_COL: lambda df: 2 + ~pd.isnull(df.relations).astype(int)}).drop(TARGET_COL, axis=1)
train_df

,text_index,e1,e2,text,label
0,3714,0,0,"Ce 6 mai 2016, les membres du ""Groupement Fina...",0
1,3714,1,0,"Ce 6 mai 2016, les <e1><GROUP_OF_INDIVIDUALS>m...",0
2,3714,1,1,"Ce 6 mai 2016, les <e1><GROUP_OF_INDIVIDUALS>m...",0
3,3714,2,0,"Ce 6 mai 2016, les membres du ""Groupement Fina...",0
4,3714,2,1,"Ce 6 mai 2016, les <e2><GROUP_OF_INDIVIDUALS>m...",0
...,...,...,...,...,...
124000,41520,13,25,"Une émeute a eu lieu aujourd'hui à Athènes, en...",0
124001,41520,14,25,"Une émeute a eu lieu aujourd'hui à Athènes, en...",0
124002,41520,15,25,"Une émeute a eu lieu aujourd'hui à Athènes, en...",0
124003,41520,16,25,"Une émeute a eu lieu aujourd'hui à Athènes, en...",0


In [5]:
get_cat_var_distribution(train_df[TASK_TARGET_COL])

,count,proportion
label,,
0,97611,0.787154
1,26394,0.212846


### train-validation split

In [6]:
VALIDATION_RATE = 0.2
train_df, val_df = train_test_split(train_df, test_size=VALIDATION_RATE, shuffle=True, random_state=RANDOM_SEED, stratify=train_df[TASK_TARGET_COL])
train_df.shape, val_df.shape

((99204, 5), (24801, 5))

In [7]:
get_cat_var_distribution(train_df[TASK_TARGET_COL])

,count,proportion
label,,
0,78089,0.787156
1,21115,0.212844


In [8]:
get_cat_var_distribution(val_df[TASK_TARGET_COL])

,count,proportion
label,,
0,19522,0.787146
1,5279,0.212854


## Create the tokenized datasets for model input

### init the tokenizer

In [9]:
# Hyperparameters
tokenizer = AutoTokenizer.from_pretrained(BASE_CHECKPOINT)
task_special_tokens = ["<e1>", "</e1>", "<e2>", "</e2>"] + [
    f"<{entity_class}>" for entity_class in entity_classes
]
# add special tokens to the tokenizer
num_added_tokens = tokenizer.add_tokens(task_special_tokens, special_tokens=True)
num_added_tokens, len(tokenizer)

/home/tagny/.pyenv/versions/3.11.9/envs/kaggle/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


(59, 119606)

### init the train-valid datasets from dataframe

In [10]:
train_ds = Dataset.from_pandas(train_df, preserve_index=False)
val_ds = Dataset.from_pandas(train_df, preserve_index=False)
task_datasets = DatasetDict({"train": train_ds, "validation": val_ds})
task_datasets

DatasetDict({
    train: Dataset({
        features: ['text_index', 'e1', 'e2', 'text', 'label'],
        num_rows: 99204
    })
    validation: Dataset({
        features: ['text_index', 'e1', 'e2', 'text', 'label'],
        num_rows: 99204
    })
})

In [11]:
task_datasets["train"][0]

{'text_index': 2569,
 'e1': 9,
 'e2': 22,
 'text': 'Cet après-midi à Sao Paulo, la famille Rodriguez a célébré le premier anniversaire de leur entreprise. L’usine, nommée “Raffinerie Rodriguez” pour perpétuer leur nom de famille, a reçu plusieurs distinctions de l’organisation “raffineries familiales prospères”. <e2><TIME_EXACT>Au beau milieu</e2> du cocktail, le directeur de l’usine, Monsieur Jean-Marin Rodriguez, a majestueusement ouvert le champagne avec un sabre. Un peu plus tard, l’alarme du détecteur de fumée s’est déclenchée. Une grande quantité de pétrole mal stocké par un employé négligeant a déclenché un incendie. Les secours ont été appelés sur le <e1><MATERIEL>téléphone fixe</e1> de la “Raffinerie Rodriguez” et le bâtiment a été évacué.',
 'label': 0}

In [12]:
task_datasets["train"][1]

{'text_index': 3736,
 'e1': 10,
 'e2': 10,
 'text': "Aujourd'hui, des centaines de personnes se sont rassemblées à Édimbourg pour protester contre la fermeture des magasins de produits exotiques. Les manifestants, principalement des immigrants, appartenaient tous à l'association « Mon Pays et Moi ». Ils défendaient le fait que ces magasins leur permettaient de retrouver des bijoux, des fruits et des vêtements de leur pays d'origine et souhaitaient donc que le gouvernement revienne sur sa décision. Selon Losseni Scott, président de l'association, tout le monde a besoin de quelque chose qui lui rappelle sa patrie. <e1><CIVILIAN>Leila Scott</e1>, membre du Sénat et sœur de Losseni Scott, était également présente parmi les manifestants. <e1><CIVILIAN>Elle</e1> a déclaré que ces magasins contribuaient également à l'équilibre financier du pays, car on pouvait y trouver des pierres précieuses ou des tapis valant des millions de livres.",
 'label': 1}

### Tokenize the datasets

In [13]:
def tokenize_function(example: dict):
    return tokenizer(example["text"], truncation=True)

# We’re using batched=True in our call to map so the function is applied to multiple elements of our dataset at once, and not on each element separately
# This is way faster
# Source https://huggingface.co/learn/nlp-course/chapter3/2?fw=pt
# columns are removed because DataCollatorWithPadding doesn't support any other columns than the ones produced by the tokenizer (or non tensors)
tokenized_datasets = task_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map: 100%|██████████| 99204/99204 [00:21<00:00, 4514.29 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text_index', 'e1', 'e2', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 99204
    })
    validation: Dataset({
        features: ['text_index', 'e1', 'e2', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 99204
    })
})

In [14]:
tokenized_datasets.column_names

{'train': ['text_index',
  'e1',
  'e2',
  'text',
  'label',
  'input_ids',
  'token_type_ids',
  'attention_mask'],
 'validation': ['text_index',
  'e1',
  'e2',
  'text',
  'label',
  'input_ids',
  'token_type_ids',
  'attention_mask']}

### Test the batch-level padding with a data collator

In [15]:

samples = tokenized_datasets.remove_columns(task_datasets["train"].column_names)["train"][:8]
samples = {k: v for k, v in samples.items()}
[len(x) for x in samples["input_ids"]]

[183, 205, 223, 112, 184, 161, 167, 185]

In [16]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 223]),
 'token_type_ids': torch.Size([8, 223]),
 'attention_mask': torch.Size([8, 223])}

## Fine-tuning a model with the Trainer API

### Compute the weight of classes to handle imbalance

In [17]:
# Source: https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#calculate_class_weights
# Scaling by total/2 helps keep the loss to a similar magnitude.
n_examples = train_df.shape[0]
n_classes = train_df[TASK_TARGET_COL].nunique()
def compute_class_weights(lbl_df: pd.DataFrame) -> pd.Series:
    return get_cat_var_distribution(lbl_df[TASK_TARGET_COL]).reset_index(drop=False)["count"].apply(lambda x: (1 / x) * (n_examples / n_classes)).rename("weight")
pd.concat([get_cat_var_distribution(train_df[TASK_TARGET_COL]), compute_class_weights(train_df)], axis=1)

,count,proportion,weight
0,78089,0.787156,0.635198
1,21115,0.212844,2.349136


### Init the model

In [18]:
model = AutoModelForSequenceClassification.from_pretrained(BASE_CHECKPOINT, num_labels=n_classes)
model.resize_token_embeddings(len(tokenizer))

/home/tagny/.pyenv/versions/3.11.9/envs/kaggle/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(119606, 768)

### Init the trainer

Source: https://stackoverflow.com/questions/69087044/early-stopping-in-bert-trainer-instances#69087153

1. Use `load_best_model_at_end = True` (EarlyStoppingCallback() requires this to be True).
2. `evaluation_strategy = 'steps'` or IntervalStrategy.STEPS instead of 'epoch'.
3. `eval_steps = 50` (evaluate the metrics after N steps).
4. `metric_for_best_model = 'f1'`

In [19]:
def compute_metrics(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average="macro")
    precision = precision_score(y_true=labels, y_pred=pred, average="macro")
    f1 = f1_score(y_true=labels, y_pred=pred, average="macro")    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

training_args = TrainingArguments(
    output_dir=os.path.join(MODELS_DIR, f"{TASK_NAME}-trainer"),
    per_device_train_batch_size=16,    
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    evaluation_strategy=IntervalStrategy.EPOCH, # steps
    eval_steps = 50, # Evaluation and Save happens every 50 steps
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy=IntervalStrategy.EPOCH,
    logging_dir=os.path.join(MODELS_DIR, f"{TASK_NAME}-trainer-tensorboard"),
    save_total_limit = 2, # Only last 2 models are saved. Older ones are deleted
    push_to_hub=False,
    metric_for_best_model = 'f1',
    load_best_model_at_end=True,
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
)

/home/tagny/.pyenv/versions/3.11.9/envs/kaggle/lib/python3.11/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


### Launch the training

In [20]:
trainer.train()

[15:56:08|ERROR|jupyter.py:224] Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tagny (ifiafrik-team). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


In [ ]:
# from datasets import load_dataset
# from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer

# raw_datasets = load_dataset("glue", "mrpc")
# checkpoint = "bert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)


# def tokenize_function(example):
#     return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


# tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# import evaluate
# def compute_metrics(eval_preds):
#     metric = evaluate.load("glue", "mrpc")
#     logits, labels = eval_preds
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

# training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
# model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# trainer = Trainer(
#     model,
#     training_args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["validation"],
#     data_collator=data_collator,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
# )

# trainer.train()